# Работен процес с участие на човек с Microsoft Agent Framework

## 🎯 Цели на обучението

В този тетрадка ще научите как да внедрите **работни процеси с участие на човек** с помощта на `RequestInfoExecutor` от Microsoft Agent Framework. Този мощен модел позволява пауза в AI работните процеси за събиране на човешки вход, правейки агентите интерактивни и предоставяйки на хората контрол върху критични решения.

## 🔄 Какво е участие на човек?

**Участие на човек (HITL)** е модел на дизайн, при който AI агентите спират изпълнението, за да поискат човешки вход преди да продължат. Това е важно за:

- ✅ **Критични решения** - Получаване на човешко одобрение преди предприемане на важни действия
- ✅ **Неясни ситуации** - Позволяване на хората да уточнят, когато AI е несигурен
- ✅ **Потребителски предпочитания** - Питане на потребителите да изберат между няколко опции
- ✅ **Съответствие и безопасност** - Осигуряване на човешки надзор за регулирани операции
- ✅ **Интерактивни преживявания** - Създаване на разговорни агенти, които реагират на вход от потребителя

## 🏗️ Как работи в Microsoft Agent Framework

Фреймуъркът предоставя три ключови компонента за HITL:

1. **`RequestInfoExecutor`** - Специален изпълнител, който спира работния процес и излъчва `RequestInfoEvent`
2. **`RequestInfoMessage`** - Основен клас за типизирани заявки, изпращани към хората
3. **`RequestResponse`** - Свързва човешките отговори с оригиналните заявки чрез `request_id`

**Модел на работния процес:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Нашият пример: Резервация на хотел с потвърждение от потребителя

Ще надградим условния работен процес, като добавим човешко потвърждение **преди** да предложим алтернативни дестинации:

1. Потребителят заявява дестинация (например "Париж")
2. `availability_agent` проверява дали има налични стаи
3. **Ако няма стаи** → `confirmation_agent` пита "Искате ли да видите алтернативи?"
4. Работният процес **спира** с помощта на `RequestInfoExecutor`
5. **Човекът отговаря** "да" или "не" чрез конзолен вход
6. `decision_manager` насочва според отговора:
   - **Да** → Показване на алтернативни дестинации
   - **Не** → Отказ на заявката за резервация
7. Показване на крайния резултат

Това демонстрира как да предоставите на потребителите контрол върху предложенията на агента!

---

Да започваме! 🚀


## Стъпка 1: Импортиране на необходимите библиотеки

Импортираме стандартните компоненти на Agent Framework плюс **специфични класове за човешко участие**:
- `RequestInfoExecutor` - Изпълнител, който спира работния процес за въвеждане от човек
- `RequestInfoEvent` - Събитие, което се излъчва, когато се изисква човешко участие
- `RequestInfoMessage` - Основен клас за типизирани заявки за данни
- `RequestResponse` - Свързва човешките отговори със заявките
- `WorkflowOutputEvent` - Събитие за откриване на изходи от работния процес


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Стъпка 2: Дефиниране на Pydantic модели за структурирани изходи

Тези модели определят **схемата**, която агентите ще връщат. Запазваме всички модели от условния работен процес и добавяме:

**Ново за участие на човек в процеса:**
- `HumanFeedbackRequest` - Подклас на `RequestInfoMessage`, който дефинира заявката, изпращана към хората
  - Съдържа `prompt` (въпрос за задаване) и `destination` (контекст за недостъпния град)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Стъпка 3: Създаване на инструмента за резервация на хотел

Същият инструмент от условния работен процес - проверява дали има налични стаи в избраната дестинация.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Стъпка 4: Дефиниране на функции за условия за маршрутизиране

Нуждаем се от **четири функции за условия** за нашия процес с човешка намеса:

**От условния работен процес:**
1. `has_availability_condition` - Маршрутизира, когато има налични хотели
2. `no_availability_condition` - Маршрутизира, когато няма налични хотели

**Нови за процеса с човешка намеса:**
3. `user_wants_alternatives_condition` - Маршрутизира, когато потребителят казва "да" на алтернативи
4. `user_declines_alternatives_condition` - Маршрутизира, когато потребителят казва "не" на алтернативи


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Стъпка 5: Създаване на изпълнител за Decision Manager

Това е **сърцевината на модела с човешко участие**! `DecisionManager` е персонализиран `Executor`, който:

1. **Получава обратна връзка от човека** чрез обекти `RequestResponse`
2. **Обработва решението на потребителя** (да/не)
3. **Насочва работния процес**, като изпраща съобщения до съответните агенти

Основни характеристики:
- Използва декоратора `@handler`, за да направи методите достъпни като стъпки в работния процес
- Получава `RequestResponse[HumanFeedbackRequest, str]`, съдържащ както оригиналната заявка, така и отговора на потребителя
- Генерира прости съобщения "да" или "не", които задействат нашите условни функции


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Стъпка 6: Създаване на персонализиран изпълнител за показване

Същият изпълнител за показване от условния работен процес - предоставя окончателни резултати като изход от работния процес.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Стъпка 7: Зареждане на променливи на средата

Конфигурирайте клиента на LLM (GitHub Models, Azure OpenAI или OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Стъпка 8: Създаване на AI агенти и изпълнители

Създаваме **шест компонента на работния процес**:

**Агенти (опаковани в AgentExecutor):**
1. **availability_agent** - Проверява наличността на хотели с помощта на инструмента
2. **confirmation_agent** - 🆕 Подготвя заявка за потвърждение от човек
3. **alternative_agent** - Предлага алтернативни градове (когато потребителят каже "да")
4. **booking_agent** - Насърчава резервация (когато има налични стаи)
5. **cancellation_agent** - 🆕 Обработва съобщение за анулиране (когато потребителят каже "не")

**Специални изпълнители:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, който спира работния процес за въвеждане от човек
7. **decision_manager** - 🆕 Персонализиран изпълнител, който насочва според отговора на човека (вече дефиниран по-горе)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Стъпка 9: Създаване на работния процес с участие на човек

Сега изграждаме графиката на работния процес с **условно маршрутизиране**, включително пътя с участие на човек:

**Структура на работния процес:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Основни връзки:**
- `availability_agent → confirmation_agent` (когато няма налични стаи)
- `confirmation_agent → prepare_human_request` (преобразуване на типа)
- `prepare_human_request → request_info_executor` (пауза за участие на човек)
- `request_info_executor → decision_manager` (винаги - предоставя RequestResponse)
- `decision_manager → alternative_agent` (когато потребителят каже "да")
- `decision_manager → cancellation_agent` (когато потребителят каже "не")
- `availability_agent → booking_agent` (когато има налични стаи)
- Всички пътища завършват при `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Стъпка 10: Изпълнение на тестов случай 1 - Град БЕЗ наличност (Париж с човешко потвърждение)

Този тест демонстрира **пълния цикъл с човешко участие**:

1. Заявка за хотел в Париж
2. availability_agent проверява → Няма свободни стаи
3. confirmation_agent създава въпрос, насочен към човек
4. request_info_executor **спира работния процес** и излъчва `RequestInfoEvent`
5. **Приложението открива събитието и подканя потребителя в конзолата**
6. Потребителят въвежда "да" или "не"
7. Приложението изпраща отговор чрез `send_responses_streaming()`
8. decision_manager насочва според отговора
9. Показва се крайният резултат

**Основен модел:**
- Използвайте `workflow.run_stream()` за първата итерация
- Използвайте `workflow.send_responses_streaming(pending_responses)` за следващите итерации
- Слушайте за `RequestInfoEvent`, за да откриете кога е необходим човешки вход
- Слушайте за `WorkflowOutputEvent`, за да уловите крайните резултати


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Стъпка 11: Изпълнение на тестов случай 2 - Град С Наличност (Стокхолм - Без нужда от човешка намеса)

Този тест демонстрира **директния път**, когато има налични стаи:

1. Заявка за хотел в Стокхолм
2. availability_agent проверява → Стаи налични ✅
3. booking_agent предлага резервация
4. display_result показва потвърждение
5. **Не е необходима човешка намеса!**

Работният процес напълно заобикаля пътя с човешко участие, когато има налични стаи.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Основни изводи и най-добри практики за човешко участие

### ✅ Какво научихте:

#### 1. **Модел RequestInfoExecutor**
Моделът за човешко участие в Microsoft Agent Framework използва три ключови компонента:
- `RequestInfoExecutor` - Спира работния процес и излъчва събития
- `RequestInfoMessage` - Основен клас за типизирани заявки (наследете го!)
- `RequestResponse` - Свързва човешките отговори с оригиналните заявки

**Критично разбиране:**
- `RequestInfoExecutor` НЕ събира входни данни сам - той само спира работния процес
- Вашият приложен код трябва да слуша за `RequestInfoEvent` и да събира входни данни
- Трябва да извикате `send_responses_streaming()` с речник, който свързва `request_id` с отговора на потребителя

#### 2. **Модел за стрийминг изпълнение**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Архитектура, базирана на събития**
Слушайте за специфични събития, за да контролирате работния процес:
- `RequestInfoEvent` - Нужен е човешки вход (работният процес е спрян)
- `WorkflowOutputEvent` - Крайният резултат е наличен (работният процес е завършен)
- `WorkflowStatusEvent` - Промени в състоянието (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS и др.)

#### 4. **Персонализирани изпълнители с @handler**
`DecisionManager` демонстрира как да създавате изпълнители, които:
- Използват декоратора `@handler`, за да изложат методи като стъпки в работния процес
- Получават типизирани съобщения (например `RequestResponse[HumanFeedbackRequest, str]`)
- Насочват работния процес, като изпращат съобщения към други изпълнители
- Достъпват контекста чрез `WorkflowContext`

#### 5. **Условно насочване с човешки решения**
Можете да създавате функции за условия, които оценяват човешките отговори:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Приложения в реалния свят:

1. **Одобрителни работни процеси**
   - Получаване на одобрение от мениджър преди обработка на разходни отчети
   - Изискване на човешки преглед преди изпращане на автоматизирани имейли
   - Потвърждение на транзакции с висока стойност преди изпълнение

2. **Модериране на съдържание**
   - Маркиране на съмнително съдържание за човешки преглед
   - Искане от модераторите да вземат окончателно решение за гранични случаи
   - Ескалиране към хора, когато увереността на AI е ниска

3. **Обслужване на клиенти**
   - Позволяване на AI да обработва рутинни въпроси автоматично
   - Ескалиране на сложни проблеми към човешки агенти
   - Искане от клиента дали иска да говори с човек

4. **Обработка на данни**
   - Искане от хората да разрешат неясни записи на данни
   - Потвърждение на интерпретациите на AI за неясни документи
   - Позволяване на потребителите да избират между няколко валидни интерпретации

5. **Системи с критична безопасност**
   - Изискване на човешко потвърждение преди необратими действия
   - Получаване на одобрение преди достъп до чувствителни данни
   - Потвърждение на решения в регулирани индустрии (здравеопазване, финанси)

6. **Интерактивни агенти**
   - Създаване на разговорни ботове, които задават последващи въпроси
   - Създаване на помощници, които водят потребителите през сложни процеси
   - Проектиране на агенти, които си сътрудничат с хора стъпка по стъпка

### 🔄 Сравнение: С и без човешко участие

| Функция | Условен работен процес | Работен процес с човешко участие |
|---------|---------------------|---------------------------|
| **Изпълнение** | Единично `workflow.run()` | Цикъл с `run_stream()` + `send_responses_streaming()` |
| **Потребителски вход** | Няма (напълно автоматизиран) | Интерактивни подкани чрез `input()` или UI |
| **Компоненти** | Агенти + Изпълнители | + RequestInfoExecutor + DecisionManager |
| **Събития** | Само AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent и др. |
| **Спиране** | Без спиране | Работният процес спира при RequestInfoExecutor |
| **Човешки контрол** | Без човешки контрол | Хората вземат ключови решения |
| **Приложение** | Автоматизация | Сътрудничество и надзор |

### 🚀 Напреднали модели:

#### Множество точки за човешки решения
Можете да имате множество `RequestInfoExecutor` възли в един и същ работен процес:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Обработка на таймаут
Реализирайте таймаути за човешки отговори:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Интеграция с богати потребителски интерфейси
Вместо `input()`, интегрирайте с уеб UI, Slack, Teams и др.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Условно човешко участие
Искайте човешки вход само в специфични ситуации:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Най-добри практики:

1. **Винаги наследявайте RequestInfoMessage**
   - Осигурява типова безопасност и валидиране
   - Позволява богат контекст за рендиране на UI
   - Уточнява намерението на всеки тип заявка

2. **Използвайте описателни подкани**
   - Включете контекст за това, което питате
   - Обяснете последствията от всяко решение
   - Поддържайте въпросите прости и ясни

3. **Обработвайте неочакван вход**
   - Валидирайте потребителските отговори
   - Осигурете стойности по подразбиране за невалиден вход
   - Давайте ясни съобщения за грешки

4. **Проследявайте идентификаторите на заявките**
   - Използвайте връзката между request_id и отговорите
   - Не се опитвайте да управлявате състоянието ръчно

5. **Проектирайте за неблокиране**
   - Не блокирайте нишки, чакащи вход
   - Използвайте асинхронни модели навсякъде
   - Поддържайте конкурентни инстанции на работния процес

### 📚 Свързани концепции:

- **Agent Middleware** - Прехващане на обаждания на агенти (предишна тетрадка)
- **Управление на състоянието на работния процес** - Запазване на състоянието на работния процес между изпълненията
- **Сътрудничество между множество агенти** - Комбиниране на човешко участие с екипи от агенти
- **Архитектури, базирани на събития** - Създаване на реактивни системи със събития

---

### 🎓 Поздравления!

Вие овладяхте работните процеси с човешко участие в Microsoft Agent Framework! Сега знаете как да:
- ✅ Спирате работни процеси за събиране на човешки вход
- ✅ Използвате RequestInfoExecutor и RequestInfoMessage
- ✅ Обработвате стрийминг изпълнение със събития
- ✅ Създавате персонализирани изпълнители с @handler
- ✅ Насочвате работни процеси въз основа на човешки решения
- ✅ Създавате интерактивни AI агенти, които си сътрудничат с хора

**Това е критичен модел за изграждане на надеждни и контролируеми AI системи!** 🚀



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за каквито и да е недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
